In [ ]:
import numpy as np
import pandas as pd
import pickle
import math
import random
import os
import matplotlib.pyplot as plt
import collections
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
import collections
from utils import get_cableidx
###################
n_f=10
n_node=106 #노드 수
N=4000 # 총 데이터 수
#split index
np.random.seed(1)
indices = np.random.permutation(N);print(indices[0])
n_train=int(N*0.6) #training 60% 
n_val=int(N*0.1) # validation 10% 
n_test=N-n_train-n_val # test 30% 
print(n_train,n_val,n_test)
train_idx, val_idx,test_idx = indices[:n_train], indices[n_train:n_train+n_val], indices[-n_test:]
sensor_no=np.array([1, 7, 9, 12, 19, 22, 28, 31, 34, 39])-1 #센서 번호
###################

In [ ]:
########## label data: 케이블의 단면적, 손상된 케이블 위치
save_flag=False#True#
if False:
    y=pd.read_csv('data/raw_data/area.csv') #경로 설정
    label_area=y.values[1:,1:]#4000,40
    for cat in ["train", "val", "test"]: 
        _l=locals()["l_" + cat] = label_area[locals()[cat+"_idx"]]
        _c=np.argmin(_l,axis=1)
        _a=np.min(_l,axis=1)
        _a=_a[:,np.newaxis]
        print(cat,_l.shape)
        if save_flag:
            np.savez_compressed('data/label/label_'+cat,_l)
            np.savez_compressed('data/label/a_label_'+cat,_a) #손상된 케이블의 단면적
            np.savez_compressed('data/label/c_label_'+cat,_c) # 손상된 케이블의 위치

In [ ]:
########## MLP data          
save_flag=False#True#
if False:#True:
    in_cable=sensor_no#np.array([1,4,9,11,16,21,24,29,31,36])-1
    with open("data/efo_signal_list.pkl", 'rb') as pickle_file:
        tension = pickle.load(pickle_file)
    tension=np.array(tension);print('whole tension',tension.shape) #(4001, 4, 40)
    tension=tension[1:,-1,in_cable]
    print('final tension',tension.shape) #(4000, 10)
    #split
    t_train, t_val, t_test = tension[train_idx],tension[val_idx], tension[test_idx]
    print(t_train.shape,t_val.shape,t_test.shape)
    #scaling
    sc=MinMaxScaler()
    t_train=sc.fit_transform(t_train)
    t_val=sc.transform(t_val)
    t_test=sc.transform(t_test)
    #save
    if save_flag:
        data_path='data/data_mlp/'
        np.savez_compressed(data_path+'t_train',t_train)
        np.savez_compressed(data_path+'t_val',t_val)
        np.savez_compressed(data_path+'t_test',t_test)

In [ ]:
########## MPNN data
num_node=10
node_loc=np.load('data/node_loc.npy')
in_cable=sensor_no#np.array([1,4,9,11,16,21,24,29,31,36])-1#list(range(3,40,4))#[3,7,11,15,19,23,27,31,35,39]-->array([ 0,  6,  8, 11, 18, 21, 27, 30, 33, 38])
#plt.scatter(node_loc[node_cable[:,0]][:,0],node_loc[node_cable[:,0]][:,1])
### node features
with open("data/efo_signal_list.pkl", 'rb') as pickle_file:
    tension = pickle.load(pickle_file)
tension=np.array(tension)#(4001, 4, 40)
tension=tension[1:,-1,in_cable] #4000,10
#indexing
t_train, t_val,t_test = tension[train_idx],tension[val_idx], tension[test_idx]
#scaling
sc=MinMaxScaler()
t_train=sc.fit_transform(t_train).reshape((-1,num_node,1))
t_val=sc.transform(t_val).reshape((-1,num_node,1))
t_test=sc.transform(t_test).reshape((-1,num_node,1))
print(t_train.shape,t_val.shape,t_test.shape)

### edge features
node_cable= (get_cableidx()[in_cable,1:])-1
node_in_cable = node_loc[node_cable[:,0]]
dist_mx=np.zeros((num_node,num_node))
for i in range(num_node):
    for j in range(num_node):
        x = float(node_in_cable[i][0]) - float(node_in_cable[j][0])
        y = float(node_in_cable[i][1]) - float(node_in_cable[j][1])
        z = float(node_in_cable[i][2]) - float(node_in_cable[j][2])
        x = math.pow(x,2)
        y = math.pow(y,2)
        z = math.pow(z,2)
        dis = math.sqrt(x+y+z)
        dist_mx[i, j] = dis
# normalization
normalized_k=0.1
distances = dist_mx[~np.isinf(dist_mx)].flatten()
std = distances.std()
dist_mx = np.exp(-np.square(dist_mx / std))
dist_mx[dist_mx < normalized_k] = 0
dist_mx=dist_mx.astype('float32')
#edge data
edge_data=[[i,j] for i,j in zip(np.where(dist_mx>0)[0],np.where(dist_mx>0)[1])];edge_data=np.array(edge_data)
edge_data=edge_data[edge_data[:,0]<edge_data[:,1]]
edge_feature=dist_mx[edge_data[:,0],edge_data[:,1]]
edge_feature=edge_feature.reshape((-1,1))

save_flag = False#True
if save_flag:
    data_path='data/data_mpnn_default_sensor/'
    np.savez_compressed(data_path+'n_train',t_train)
    np.savez_compressed(data_path+'n_val',t_val)
    np.savez_compressed(data_path+'n_test',t_test)
    np.savez_compressed(data_path+'edge_feature',edge_feature)
    np.savez_compressed(data_path+'edge_data',edge_data)